# Output Widget append_display_data Test

This notebook tests the functionality of `append_display_data()` on Output widgets.
The issue described in [#16739](https://github.com/microsoft/vscode-jupyter/issues/16739) is that content added asynchronously via `append_display_data()` doesn't appear in VS Code.

In [ ]:
import asyncio
from IPython.display import display, HTML
from ipywidgets import Output

## Test 1: Synchronous append_display_data

According to @DonJayamanne's comment, even synchronous calls don't work.

In [ ]:
# Create output widgets
sync_outputs = [Output() for _ in range(2)]
display(*sync_outputs)

In [ ]:
# Append content synchronously
sync_outputs[0].append_display_data(HTML("<div style='color: green; font-weight: bold;'>✅ Sync Content 0 - This should appear!</div>"))
sync_outputs[1].append_display_data(HTML("<div style='color: blue; font-weight: bold;'>✅ Sync Content 1 - This should appear!</div>"))

## Test 2: Asynchronous append_display_data

This was the original issue - async content doesn't appear.

In [ ]:
# Create output widgets for async test
async_outputs = [Output() for _ in range(3)]
display(*async_outputs)

async def append_async_content():
    for i in range(3):
        await asyncio.sleep(0.1)  # Small delay
        color = ['red', 'orange', 'purple'][i]
        async_outputs[i].append_display_data(
            HTML(f"<div style='color: {color}; font-weight: bold;'>✅ Async Content {i} - This should appear!</div>")
        )

# Start the async task
asyncio.create_task(append_async_content())

## Test 3: Mixed content types

Test with different content types (HTML, text, etc.)

In [ ]:
# Create output widget for mixed content
mixed_output = Output()
display(mixed_output)

# Add different types of content
mixed_output.append_display_data(HTML("<h3>HTML Content</h3>"))
mixed_output.append_display_data({"text/plain": "Plain text content"})
mixed_output.append_display_data(HTML("<p style='color: green;'>More HTML content</p>"))

## Expected Results

- **Before fix**: Content doesn't appear in Output widgets, console shows "Model not found in Kernel state to render output" errors
- **After fix**: All content should appear properly in the respective Output widgets

If you see the colored text content in the Output widgets above, the fix is working!